# Análisis Avanzado - CoinMarketCap Data

---

## 📊 Introducción
Este notebook contiene análisis avanzados sobre los datos extraídos del pipeline ETL de CoinMarketCap.

**Pre-requisitos:**
- Base de datos `coinmarketcap_etl.db` creada por el notebook ETL principal
- Tablas disponibles: `crypto_listings`, `global_metrics`, `crypto_metadata`

---

## 🎯 Objetivos del Análisis
1. Explorar patrones y tendencias en el mercado de criptomonedas
2. Crear métricas derivadas y ratios analíticos
3. Segmentar el mercado por diferentes criterios
4. Generar tablas analíticas para consultas rápidas

---

## 👨‍💻 Autor
**Mario Soriano Bañuls** - Portfolio profesional 

In [1]:
# Importar librerías necesarias
import sqlite3
import pandas as pd
from datetime import datetime

# Configuración
DB_PATH = 'coinmarketcap_etl.db'

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [2]:
# Función auxiliar para consultas SQL
def consulta_sqlite(query, db_path='coinmarketcap_etl.db'):
    """
    Ejecuta una consulta SQL y devuelve un DataFrame de pandas.
    
    Args:
        query (str): Consulta SQL a ejecutar
        db_path (str): Ruta a la base de datos SQLite
    
    Returns:
        pd.DataFrame: Resultados de la consulta
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"❌ Error en consulta SQL: {e}")
        return None

print("✅ Función consulta_sqlite lista")

✅ Función consulta_sqlite lista


---

## 🔍 1. Verificación de Estructura de Datos

Exploramos las tablas disponibles y sus esquemas.

In [3]:
# Verificar estructura de tablas actuales
conn = sqlite3.connect(DB_PATH)

print("📋 ESTRUCTURA DE TABLAS DISPONIBLES\n")

for tabla in ['crypto_listings', 'global_metrics', 'crypto_metadata']:
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({tabla})")
    columnas = cursor.fetchall()
    
    print(f"\n📊 {tabla.upper()}")
    print("-" * 60)
    for col in columnas:
        print(f"  • {col[1]} ({col[2]})")
    
    cursor.execute(f"SELECT COUNT(*) FROM {tabla}")
    count = cursor.fetchone()[0]
    print(f"  Total registros: {count}")

conn.close()

📋 ESTRUCTURA DE TABLAS DISPONIBLES


📊 CRYPTO_LISTINGS
------------------------------------------------------------
  • id (INTEGER)
  • name (TEXT)
  • symbol (TEXT)
  • slug (TEXT)
  • cmc_rank (INTEGER)
  • num_market_pairs (INTEGER)
  • circulating_supply (REAL)
  • total_supply (REAL)
  • max_supply (REAL)
  • price (REAL)
  • volume_24h (REAL)
  • volume_change_24h (REAL)
  • percent_change_1h (REAL)
  • percent_change_24h (REAL)
  • percent_change_7d (REAL)
  • market_cap (REAL)
  • market_cap_dominance (REAL)
  • market_cap_category (TEXT)
  Total registros: 100

📊 GLOBAL_METRICS
------------------------------------------------------------
  • total_cryptocurrencies (INTEGER)
  • active_cryptocurrencies (INTEGER)
  • total_exchanges (INTEGER)
  • active_exchanges (INTEGER)
  • active_market_pairs (INTEGER)
  • total_market_cap (REAL)
  • total_volume_24h (REAL)
  • btc_dominance (REAL)
  • eth_dominance (REAL)
  • defi_volume_24h (REAL)
  • defi_market_cap (REAL)
  Total regist

---

## 📈 2. Análisis de Top Movers

Identificamos las criptomonedas con mayores cambios (positivos y negativos) en las últimas 24 horas.

In [4]:
# ANÁLISIS 1: Top Movers (Mayores ganadores y perdedores)

query_top_movers = """
WITH ranked_cryptos AS (
    SELECT 
        name,
        symbol,
        price,
        percent_change_24h,
        volume_24h,
        market_cap,
        CASE 
            WHEN percent_change_24h > 0 THEN 'Ganador'
            ELSE 'Perdedor'
        END as tipo
    FROM crypto_listings
    WHERE percent_change_24h IS NOT NULL
)
SELECT 
    tipo,
    name,
    symbol,
    ROUND(price, 4) as precio_usd,
    ROUND(percent_change_24h, 2) as cambio_24h_pct,
    ROUND(volume_24h / 1000000, 2) as volumen_24h_millones,
    ROUND(market_cap / 1000000000, 2) as market_cap_billones
FROM ranked_cryptos
ORDER BY ABS(percent_change_24h) DESC
LIMIT 10
"""

df_movers = consulta_sqlite(query_top_movers)
if df_movers is not None:
    print("\n🎯 TOP 10 MOVERS (Mayores cambios en 24h)\n")
    display(df_movers)


🎯 TOP 10 MOVERS (Mayores cambios en 24h)



,tipo,name,symbol,precio_usd,cambio_24h_pct,volumen_24h_millones,market_cap_billones
0,Ganador,MYX Finance,MYX,3.2706,21.54,54.72,0.67
1,Ganador,Virtuals Protocol,VIRTUAL,0.9035,14.63,314.84,0.59
2,Ganador,Zcash,ZEC,255.4646,6.99,576.50,4.16
3,Perdedor,World Liberty Financial,WLFI,0.1358,-6.67,288.53,3.34
4,Ganador,Pump.fun,PUMP,0.0041,6.52,267.01,1.45
5,Ganador,Artificial Superintelligence Alliance,FET,0.2637,5.05,127.84,0.62
6,Ganador,Jupiter,JUP,0.3774,4.79,59.99,1.19
7,Perdedor,PancakeSwap,CAKE,2.6326,-4.61,181.56,0.90
8,Perdedor,Sky,SKY,0.0578,-3.76,14.61,1.35
9,Perdedor,Mantle,MNT,1.6358,-3.74,190.54,5.32


---

## 💰 3. Segmentación por Market Cap

Analizamos las diferencias entre diferentes categorías de market cap.

In [5]:
# ANÁLISIS 2: Segmentación por Market Cap

query_segmentacion = """
SELECT 
    market_cap_category,
    COUNT(*) as cantidad_cryptos,
    ROUND(AVG(price), 4) as precio_promedio,
    ROUND(AVG(percent_change_24h), 2) as cambio_promedio_24h,
    ROUND(SUM(market_cap) / 1000000000, 2) as market_cap_total_billones,
    ROUND(AVG(volume_24h) / 1000000, 2) as volumen_promedio_millones
FROM crypto_listings
WHERE market_cap_category IS NOT NULL
GROUP BY market_cap_category
ORDER BY 
    CASE market_cap_category
        WHEN 'Mega-cap' THEN 1
        WHEN 'Large-cap' THEN 2
        WHEN 'Mid-cap' THEN 3
        WHEN 'Small-cap' THEN 4
    END
"""

df_segmentacion = consulta_sqlite(query_segmentacion)
if df_segmentacion is not None:
    print("\n📊 SEGMENTACIÓN POR MARKET CAP\n")
    display(df_segmentacion)


📊 SEGMENTACIÓN POR MARKET CAP



,market_cap_category,cantidad_cryptos,precio_promedio,cambio_promedio_24h,market_cap_total_billones,volumen_promedio_millones
0,Mega-cap,14,8273.6188,-0.24,3471.65,17328.74
1,Large-cap,57,180.4388,-0.00,178.62,282.91
2,Mid-cap,29,4.2703,1.23,20.86,72.80


---

## 🔢 4. Ratios y Métricas Derivadas

Calculamos indicadores avanzados como el ratio Volume/Market Cap y momentum diario.

In [6]:
# ANÁLISIS 3: Ratios y Métricas Derivadas

query_ratios = """
SELECT 
    name,
    symbol,
    ROUND(price, 4) as precio,
    ROUND(volume_24h / NULLIF(market_cap, 0), 4) as volume_to_mcap_ratio,
    ROUND((percent_change_24h - percent_change_7d) / 7, 2) as momentum_diario,
    ROUND(volume_24h / 1000000, 2) as volumen_millones,
    market_cap_category as categoria
FROM crypto_listings
WHERE market_cap > 0 
    AND volume_24h > 0
ORDER BY volume_to_mcap_ratio DESC
LIMIT 15
"""

df_ratios = consulta_sqlite(query_ratios)
if df_ratios is not None:
    print("\n📈 MÉTRICAS DERIVADAS (Top 15 por Volume/MarketCap Ratio)\n")
    display(df_ratios)
    print("\n💡 Volume/MarketCap Ratio alto = Mayor liquidez relativa")


📈 MÉTRICAS DERIVADAS (Top 15 por Volume/MarketCap Ratio)



,name,symbol,precio,volume_to_mcap_ratio,momentum_diario,volumen_millones,categoria
0,First Digital USD,FDUSD,0.9980,5.7902,-0.01,5806.73,Large-cap
1,Tether USDt,USDT,1.0003,0.6741,0.00,123315.20,Mega-cap
2,Virtuals Protocol,VIRTUAL,0.9035,0.5314,-0.76,314.84,Mid-cap
3,Plasma,XPL,0.3799,0.4045,1.41,276.59,Mid-cap
4,Aster,ASTER,1.0848,0.3097,0.70,677.87,Large-cap
5,PAX Gold,PAXG,4099.5394,0.2754,0.36,374.60,Large-cap
6,USDC,USDC,1.0000,0.2057,0.00,15691.06,Mega-cap
7,Artificial Superintelligence Alliance,FET,0.2637,0.2046,1.12,127.84,Mid-cap
8,PancakeSwap,CAKE,2.6326,0.2016,0.29,181.56,Mid-cap
9,OFFICIAL TRUMP,TRUMP,5.8836,0.1892,-0.17,222.69,Large-cap



💡 Volume/MarketCap Ratio alto = Mayor liquidez relativa


---

## 👑 5. Dominancia de Mercado

Análisis del porcentaje de dominancia que tiene cada criptomoneda sobre el mercado total.

In [7]:
# ANÁLISIS 4: Dominancia de Mercado con Metadata

query_dominancia = """
SELECT 
    l.name,
    l.symbol,
    ROUND(l.market_cap / 1000000000, 2) as mcap_billones,
    ROUND((l.market_cap / g.total_market_cap) * 100, 2) as dominancia_pct,
    m.category,
    m.platform
FROM crypto_listings l
CROSS JOIN global_metrics g
LEFT JOIN crypto_metadata m ON l.symbol = m.symbol
WHERE l.cmc_rank <= 20
ORDER BY l.market_cap DESC
"""

df_dominancia = consulta_sqlite(query_dominancia)
if df_dominancia is not None:
    print("\n👑 DOMINANCIA DE MERCADO (Top 20)\n")
    display(df_dominancia)


👑 DOMINANCIA DE MERCADO (Top 20)



,name,symbol,mcap_billones,dominancia_pct,category,platform
0,Bitcoin,BTC,2204.43,59.19,coin,None
1,Ethereum,ETH,471.93,12.67,coin,None
2,Tether USDt,USDT,182.94,4.91,token,Ethereum
3,BNB,BNB,154.27,4.14,None,None
4,XRP,XRP,148.78,3.99,None,None
5,Solana,SOL,105.37,2.83,None,None
6,USDC,USDC,76.29,2.05,None,None
7,Dogecoin,DOGE,29.69,0.80,None,None
8,TRON,TRX,28.80,0.77,None,None
9,Cardano,ADA,23.29,0.63,None,None


---

## 🗄️ 6. Crear Tabla Analítica Derivada

Generamos una nueva tabla `crypto_analytics` con métricas calculadas y categorizaciones automáticas.

In [8]:
# CREAR TABLA DERIVADA: crypto_analytics (métricas calculadas)

def crear_tabla_analytics():
    """
    Crea una tabla con métricas analíticas derivadas de los datos base.
    """
    
    query_analytics = """
    CREATE TABLE IF NOT EXISTS crypto_analytics AS
    SELECT 
        l.id,
        l.name,
        l.symbol,
        l.cmc_rank,
        l.price,
        l.market_cap,
        l.volume_24h,
        l.market_cap_category,
        
        -- Ratios
        ROUND(l.volume_24h / NULLIF(l.market_cap, 0), 4) as volume_to_mcap_ratio,
        
        -- Dominancia
        ROUND((l.market_cap / g.total_market_cap) * 100, 4) as market_dominance_pct,
        
        -- Momentum
        l.percent_change_1h,
        l.percent_change_24h,
        l.percent_change_7d,
        ROUND((l.percent_change_24h - l.percent_change_7d) / 7, 2) as daily_momentum,
        
        -- Clasificaciones
        CASE 
            WHEN l.percent_change_24h > 10 THEN 'Strong Gain'
            WHEN l.percent_change_24h > 5 THEN 'Moderate Gain'
            WHEN l.percent_change_24h > 0 THEN 'Slight Gain'
            WHEN l.percent_change_24h > -5 THEN 'Slight Loss'
            WHEN l.percent_change_24h > -10 THEN 'Moderate Loss'
            ELSE 'Strong Loss'
        END as performance_category,
        
        CASE 
            WHEN l.volume_24h / NULLIF(l.market_cap, 0) > 0.5 THEN 'Very High Liquidity'
            WHEN l.volume_24h / NULLIF(l.market_cap, 0) > 0.2 THEN 'High Liquidity'
            WHEN l.volume_24h / NULLIF(l.market_cap, 0) > 0.1 THEN 'Moderate Liquidity'
            ELSE 'Low Liquidity'
        END as liquidity_category,
        
        -- Timestamp
        datetime('now') as analyzed_at
        
    FROM crypto_listings l
    CROSS JOIN global_metrics g
    WHERE l.market_cap > 0 AND l.volume_24h > 0
    """
    
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Eliminar tabla si existe (para recrear con datos frescos)
    cursor.execute("DROP TABLE IF EXISTS crypto_analytics")
    
    # Crear nueva tabla
    cursor.execute(query_analytics)
    conn.commit()
    
    # Verificar
    cursor.execute("SELECT COUNT(*) FROM crypto_analytics")
    count = cursor.fetchone()[0]
    
    conn.close()
    
    print(f"✅ Tabla 'crypto_analytics' creada con {count} registros")
    return count

# Ejecutar
crear_tabla_analytics()

✅ Tabla 'crypto_analytics' creada con 100 registros


100

In [9]:
# Consultar la nueva tabla de analytics

query_ver_analytics = """
SELECT 
    name,
    symbol,
    ROUND(price, 4) as precio,
    market_cap_category,
    performance_category,
    liquidity_category,
    ROUND(market_dominance_pct, 2) as dominancia_pct,
    ROUND(volume_to_mcap_ratio, 4) as vol_mcap_ratio
FROM crypto_analytics
ORDER BY market_dominance_pct DESC
LIMIT 20
"""

df_analytics = consulta_sqlite(query_ver_analytics)
if df_analytics is not None:
    print("\n📊 TABLA CRYPTO_ANALYTICS (Top 20 por dominancia)\n")
    display(df_analytics)


📊 TABLA CRYPTO_ANALYTICS (Top 20 por dominancia)



,name,symbol,precio,market_cap_category,performance_category,liquidity_category,dominancia_pct,vol_mcap_ratio
0,Bitcoin,BTC,110556.7605,Mega-cap,Slight Loss,Low Liquidity,59.19,0.0225
1,Ethereum,ETH,3910.0062,Mega-cap,Slight Gain,Low Liquidity,12.67,0.0745
2,Tether USDt,USDT,1.0003,Mega-cap,Slight Gain,Very High Liquidity,4.91,0.6741
3,BNB,BNB,1108.4156,Mega-cap,Slight Loss,Low Liquidity,4.14,0.0243
4,XRP,XRP,2.4789,Mega-cap,Slight Gain,Low Liquidity,3.99,0.0277
5,Solana,SOL,191.7677,Mega-cap,Slight Loss,Low Liquidity,2.83,0.0578
6,USDC,USDC,1.0000,Mega-cap,Slight Gain,High Liquidity,2.05,0.2057
7,Dogecoin,DOGE,0.1960,Mega-cap,Slight Loss,Low Liquidity,0.80,0.0549
8,TRON,TRX,0.3043,Mega-cap,Slight Loss,Low Liquidity,0.77,0.0358
9,Cardano,ADA,0.6497,Mega-cap,Slight Loss,Low Liquidity,0.63,0.0371


---

## 🕐 7. Implementación: Historial Time-Series

Creamos una tabla que mantiene snapshots históricos de los precios.

In [10]:
# Crear tabla de historial time-series

def crear_tabla_historica():
    """
    Crea una tabla para almacenar snapshots históricos de crypto_listings.
    Esta función debe ejecutarse cada vez que se actualiza el ETL.
    """
    
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Crear tabla si no existe (con APPEND, no REPLACE)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS crypto_history (
            id INTEGER,
            name TEXT,
            symbol TEXT,
            slug TEXT,
            cmc_rank INTEGER,
            price REAL,
            volume_24h REAL,
            market_cap REAL,
            percent_change_1h REAL,
            percent_change_24h REAL,
            percent_change_7d REAL,
            market_cap_category TEXT,
            snapshot_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (symbol, snapshot_time)
        )
    """)
    
    # Insertar snapshot actual
    cursor.execute("""
        INSERT OR IGNORE INTO crypto_history 
        SELECT 
            id, name, symbol, slug, cmc_rank, 
            price, volume_24h, market_cap,
            percent_change_1h, percent_change_24h, percent_change_7d,
            market_cap_category,
            datetime('now') as snapshot_time
        FROM crypto_listings
    """)
    
    conn.commit()
    
    # Verificar
    cursor.execute("SELECT COUNT(*) FROM crypto_history")
    total = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(DISTINCT snapshot_time) FROM crypto_history")
    snapshots = cursor.fetchone()[0]
    
    conn.close()
    
    print(f"✅ Tabla crypto_history actualizada")
    print(f"   Total registros: {total}")
    print(f"   Snapshots únicos: {snapshots}")
    
    return total, snapshots

# Ejecutar
crear_tabla_historica()

✅ Tabla crypto_history actualizada
   Total registros: 100
   Snapshots únicos: 1


(100, 1)

In [11]:
# Analizar evolución de precios (Time-Series)

query_evolucion = """
SELECT 
    symbol,
    name,
    snapshot_time,
    ROUND(price, 4) as precio,
    ROUND(percent_change_24h, 2) as cambio_24h
FROM crypto_history
WHERE symbol IN ('BTC', 'ETH', 'BNB', 'SOL', 'XRP')
ORDER BY symbol, snapshot_time DESC
"""

df_evolucion = consulta_sqlite(query_evolucion)
if df_evolucion is not None:
    print("\n📈 EVOLUCIÓN DE PRECIOS (Top 5 Cryptos)\\n")
    for symbol in ['BTC', 'ETH', 'BNB', 'SOL', 'XRP']:
        df_crypto = df_evolucion[df_evolucion['symbol'] == symbol]
        if not df_crypto.empty:
            print(f"\n{symbol} - {df_crypto['name'].iloc[0]}")
            print(df_crypto[['snapshot_time', 'precio', 'cambio_24h']].to_string(index=False))
            print("-" * 60)


📈 EVOLUCIÓN DE PRECIOS (Top 5 Cryptos)\n

BTC - Bitcoin
      snapshot_time      precio  cambio_24h
2025-10-24 18:30:36 110556.7605       -0.28
------------------------------------------------------------

ETH - Ethereum
      snapshot_time    precio  cambio_24h
2025-10-24 18:30:36 3910.0062         0.3
------------------------------------------------------------

BNB - BNB
      snapshot_time    precio  cambio_24h
2025-10-24 18:30:36 1108.4156       -1.55
------------------------------------------------------------

SOL - Solana
      snapshot_time   precio  cambio_24h
2025-10-24 18:30:36 191.7677       -0.08
------------------------------------------------------------

XRP - XRP
      snapshot_time  precio  cambio_24h
2025-10-24 18:30:36  2.4789        2.74
------------------------------------------------------------


---

## 🚨 8. Sistema de Alertas Automáticas

Detectamos movimientos significativos y generamos alertas.

In [12]:
# Sistema de alertas por volatilidad extrema

def generar_alertas(umbral_cambio=10):
    """
    Genera alertas para cryptos con cambios extremos.
    
    Args:
        umbral_cambio (float): Porcentaje mínimo de cambio para alertar (default: 10%)
    """
    
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Crear tabla de alertas si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS price_alerts (
            alert_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            symbol TEXT,
            price REAL,
            percent_change_24h REAL,
            market_cap REAL,
            alert_type TEXT,
            alert_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Insertar nuevas alertas
    cursor.execute(f"""
        INSERT INTO price_alerts (name, symbol, price, percent_change_24h, market_cap, alert_type)
        SELECT 
            name, 
            symbol, 
            price, 
            percent_change_24h,
            market_cap,
            CASE 
                WHEN percent_change_24h >= {umbral_cambio} THEN 'PUMP'
                WHEN percent_change_24h <= -{umbral_cambio} THEN 'DUMP'
            END as alert_type
        FROM crypto_listings
        WHERE ABS(percent_change_24h) >= {umbral_cambio}
    """)
    
    conn.commit()
    
    # Mostrar alertas recientes
    cursor.execute("""
        SELECT 
            alert_type,
            name,
            symbol,
            ROUND(price, 4) as precio,
            ROUND(percent_change_24h, 2) as cambio_pct,
            alert_time
        FROM price_alerts
        WHERE date(alert_time) = date('now')
        ORDER BY ABS(percent_change_24h) DESC
    """)
    
    alertas = cursor.fetchall()
    conn.close()
    
    if alertas:
        print(f"\n🚨 ALERTAS GENERADAS (Umbral: ±{umbral_cambio}%)\\n")
        print(f"{'Tipo':<6} {'Nombre':<20} {'Symbol':<8} {'Precio':<12} {'Cambio %':<10} {'Hora'}")
        print("-" * 80)
        for alerta in alertas:
            tipo_emoji = "📈" if alerta[0] == "PUMP" else "📉"
            print(f"{tipo_emoji} {alerta[0]:<5} {alerta[1]:<20} {alerta[2]:<8} ${alerta[3]:<11} {alerta[4]:>7}% {alerta[5]}")
    else:
        print(f"✅ No hay alertas hoy (umbral: ±{umbral_cambio}%)")
    
    return len(alertas)

# Ejecutar con umbral de 10%
generar_alertas(umbral_cambio=10)


🚨 ALERTAS GENERADAS (Umbral: ±10%)\n
Tipo   Nombre               Symbol   Precio       Cambio %   Hora
--------------------------------------------------------------------------------
📈 PUMP  MYX Finance          MYX      $3.2706        21.54% 2025-10-24 18:30:36
📈 PUMP  Virtuals Protocol    VIRTUAL  $0.9035        14.63% 2025-10-24 18:30:36


2

---

## 💼 9. Portfolio Tracking

Crea y rastrea tu propio portfolio de criptomonedas.

In [13]:
# Crear tabla de portfolio y calcular P&L

def crear_portfolio():
    """
    Crea una tabla para trackear tu portfolio personal.
    """
    
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Crear tabla
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS my_portfolio (
            symbol TEXT PRIMARY KEY,
            nombre TEXT,
            cantidad REAL NOT NULL,
            precio_compra REAL NOT NULL,
            fecha_compra DATE NOT NULL,
            notas TEXT
        )
    """)
    
    # Insertar posiciones de ejemplo (puedes modificar o borrar después)
    posiciones_ejemplo = [
        ('BTC', 'Bitcoin', 0.5, 30000.0, '2024-01-15', 'Compra inicial'),
        ('ETH', 'Ethereum', 5.0, 2000.0, '2024-02-20', 'DCA mensual'),
        ('SOL', 'Solana', 50.0, 100.0, '2024-03-10', 'Especulación'),
    ]
    
    cursor.executemany("""
        INSERT OR IGNORE INTO my_portfolio (symbol, nombre, cantidad, precio_compra, fecha_compra, notas)
        VALUES (?, ?, ?, ?, ?, ?)
    """, posiciones_ejemplo)
    
    conn.commit()
    conn.close()
    
    print("✅ Tabla my_portfolio creada con posiciones de ejemplo")
    print("   Nota: Modifica estas posiciones según tu portfolio real")

# Ejecutar
crear_portfolio()

✅ Tabla my_portfolio creada con posiciones de ejemplo
   Nota: Modifica estas posiciones según tu portfolio real


In [14]:
# Calcular rendimiento del portfolio

query_portfolio = """
SELECT 
    p.symbol,
    p.nombre,
    p.cantidad,
    p.precio_compra,
    l.price as precio_actual,
    p.fecha_compra,
    
    -- Cálculos financieros
    ROUND(p.cantidad * p.precio_compra, 2) as inversion_inicial,
    ROUND(p.cantidad * l.price, 2) as valor_actual,
    ROUND((p.cantidad * l.price) - (p.cantidad * p.precio_compra), 2) as ganancia_perdida,
    ROUND((((l.price - p.precio_compra) / p.precio_compra) * 100), 2) as roi_pct,
    
    -- Días de tenencia
    CAST((julianday('now') - julianday(p.fecha_compra)) AS INTEGER) as dias_tenencia,
    
    p.notas
    
FROM my_portfolio p
JOIN crypto_listings l ON p.symbol = l.symbol
ORDER BY ganancia_perdida DESC
"""

df_portfolio = consulta_sqlite(query_portfolio)
if df_portfolio is not None:
    print("\n💼 RENDIMIENTO DEL PORTFOLIO\\n")
    display(df_portfolio)
    
    # Resumen
    total_inversion = df_portfolio['inversion_inicial'].sum()
    total_actual = df_portfolio['valor_actual'].sum()
    total_ganancia = df_portfolio['ganancia_perdida'].sum()
    roi_total = ((total_actual - total_inversion) / total_inversion) * 100
    
    print(f"\n📊 RESUMEN TOTAL")
    print(f"   Inversión inicial: ${total_inversion:,.2f}")
    print(f"   Valor actual: ${total_actual:,.2f}")
    print(f"   Ganancia/Pérdida: ${total_ganancia:,.2f}")
    print(f"   ROI: {roi_total:.2f}%")
    
    emoji_resultado = "📈" if total_ganancia > 0 else "📉"
    print(f"   {emoji_resultado} {'¡En ganancias!' if total_ganancia > 0 else 'En pérdidas'}")


💼 RENDIMIENTO DEL PORTFOLIO\n


,symbol,nombre,cantidad,precio_compra,precio_actual,fecha_compra,inversion_inicial,valor_actual,ganancia_perdida,roi_pct,dias_tenencia,notas
0,BTC,Bitcoin,0.5,30000.0,110556.760531,2024-01-15,15000.0,55278.38,40278.38,268.52,648,Compra inicial
1,ETH,Ethereum,5.0,2000.0,3910.006166,2024-02-20,10000.0,19550.03,9550.03,95.50,612,DCA mensual
2,SOL,Solana,50.0,100.0,191.767680,2024-03-10,5000.0,9588.38,4588.38,91.77,593,Especulación



📊 RESUMEN TOTAL
   Inversión inicial: $30,000.00
   Valor actual: $84,416.79
   Ganancia/Pérdida: $54,416.79
   ROI: 181.39%
   📈 ¡En ganancias!


---

## 📊 10. Análisis por Categorías/Tags

Comparamos rendimiento entre diferentes sectores del mercado crypto.

In [15]:
# Análisis por categorías de metadata

query_categorias = """
SELECT 
    COALESCE(m.category, 'Sin categoría') as categoria,
    COUNT(DISTINCT l.symbol) as num_cryptos,
    ROUND(AVG(l.percent_change_24h), 2) as cambio_promedio_24h,
    ROUND(SUM(l.market_cap) / 1000000000, 2) as market_cap_total_billones,
    ROUND(AVG(l.price), 4) as precio_promedio,
    ROUND(AVG(l.volume_24h) / 1000000, 2) as volumen_promedio_millones
FROM crypto_listings l
LEFT JOIN crypto_metadata m ON l.symbol = m.symbol
GROUP BY categoria
HAVING num_cryptos >= 1
ORDER BY market_cap_total_billones DESC
LIMIT 15
"""

df_categorias = consulta_sqlite(query_categorias)
if df_categorias is not None:
    print("\n📊 RENDIMIENTO POR CATEGORÍA (Top 15)\\n")
    display(df_categorias)
    
    # Identificar mejor y peor sector
    mejor_sector = df_categorias.loc[df_categorias['cambio_promedio_24h'].idxmax()]
    peor_sector = df_categorias.loc[df_categorias['cambio_promedio_24h'].idxmin()]
    
    print(f"\n🏆 Mejor sector: {mejor_sector['categoria']} (+{mejor_sector['cambio_promedio_24h']}%)")
    print(f"📉 Peor sector: {peor_sector['categoria']} ({peor_sector['cambio_promedio_24h']}%)")


📊 RENDIMIENTO POR CATEGORÍA (Top 15)\n


,categoria,num_cryptos,cambio_promedio_24h,market_cap_total_billones,precio_promedio,volumen_promedio_millones
0,coin,2,0.01,2676.36,57233.3833,42336.31
1,Sin categoría,97,0.33,811.83,121.3582,544.86
2,token,1,0.00,182.94,1.0003,123315.20



🏆 Mejor sector: Sin categoría (+0.33%)
📉 Peor sector: token (0.0%)


---

## 🔗 11. Análisis de Correlación

Identifica qué criptomonedas se mueven juntas (requiere datos históricos).

In [16]:
# Matriz de correlación de precios (requiere múltiples snapshots)

import numpy as np

def calcular_correlaciones(top_n=10):
    """
    Calcula la matriz de correlación entre las top N criptomonedas.
    Requiere al menos 2 snapshots en crypto_history.
    """
    
    # Verificar si hay suficientes datos
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(DISTINCT snapshot_time) FROM crypto_history")
    num_snapshots = cursor.fetchone()[0]
    conn.close()
    
    if num_snapshots < 2:
        print(f"⚠️  Se necesitan al menos 2 snapshots históricos")
        print(f"   Actualmente tienes: {num_snapshots}")
        print(f"   Ejecuta crear_tabla_historica() en diferentes momentos para acumular datos")
        return None
    
    # Obtener datos
    query = f"""
    SELECT symbol, snapshot_time, price
    FROM crypto_history
    WHERE symbol IN (
        SELECT symbol 
        FROM crypto_listings 
        ORDER BY market_cap DESC 
        LIMIT {top_n}
    )
    ORDER BY symbol, snapshot_time
    """
    
    df_history = consulta_sqlite(query)
    
    if df_history is None or df_history.empty:
        print("❌ No hay datos históricos disponibles")
        return None
    
    # Crear tabla pivote
    df_pivot = df_history.pivot(index='snapshot_time', columns='symbol', values='price')
    
    # Calcular correlación
    correlation_matrix = df_pivot.corr()
    
    print(f"\n🔗 MATRIZ DE CORRELACIÓN (Top {top_n} Cryptos)")
    print(f"   Basado en {num_snapshots} snapshots históricos\\n")
    display(correlation_matrix.round(3))
    
    # Encontrar pares altamente correlacionados
    print("\\n📊 PARES MÁS CORRELACIONADOS:")
    correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            correlations.append({
                'par': f"{correlation_matrix.columns[i]}-{correlation_matrix.columns[j]}",
                'correlacion': correlation_matrix.iloc[i, j]
            })
    
    df_corr = pd.DataFrame(correlations).sort_values('correlacion', ascending=False)
    print(df_corr.head(5).to_string(index=False))
    
    return correlation_matrix

# Ejecutar
calcular_correlaciones(top_n=10)

⚠️  Se necesitan al menos 2 snapshots históricos
   Actualmente tienes: 1
   Ejecuta crear_tabla_historica() en diferentes momentos para acumular datos


---

## 📉 12. Indicadores Técnicos Básicos

Calculamos indicadores técnicos simples con los datos disponibles.

In [17]:
# Indicadores técnicos simples (RSI aproximado, momentum)

def calcular_indicadores_tecnicos():
    """
    Calcula indicadores técnicos básicos basados en cambios de precio.
    """
    
    query_indicadores = """
    SELECT 
        symbol,
        name,
        ROUND(price, 4) as precio_actual,
        percent_change_1h,
        percent_change_24h,
        percent_change_7d,
        
        -- RSI Simplificado (basado en cambios recientes)
        CASE 
            WHEN percent_change_24h > 10 THEN 'Sobrecomprado'
            WHEN percent_change_24h < -10 THEN 'Sobrevendido'
            ELSE 'Neutral'
        END as rsi_signal,
        
        -- Momentum (aceleración del cambio)
        ROUND(percent_change_24h - (percent_change_7d / 7), 2) as momentum_diario,
        
        -- Tendencia general
        CASE 
            WHEN percent_change_7d > 5 THEN 'Alcista Fuerte'
            WHEN percent_change_7d > 0 THEN 'Alcista'
            WHEN percent_change_7d > -5 THEN 'Bajista'
            ELSE 'Bajista Fuerte'
        END as tendencia_7d,
        
        -- Volatilidad (dispersión entre cambios)
        ROUND(ABS(percent_change_1h - percent_change_24h), 2) as volatilidad_index
        
    FROM crypto_listings
    WHERE cmc_rank <= 20
    ORDER BY market_cap DESC
    """
    
    df_indicadores = consulta_sqlite(query_indicadores)
    
    if df_indicadores is not None:
        print("\n📉 INDICADORES TÉCNICOS (Top 20 por Market Cap)\\n")
        display(df_indicadores)
        
        # Resumen de señales
        print("\\n📊 RESUMEN DE SEÑALES:")
        print(df_indicadores['rsi_signal'].value_counts().to_string())
        print("\\n📈 TENDENCIAS:")
        print(df_indicadores['tendencia_7d'].value_counts().to_string())
    
    return df_indicadores

# Ejecutar
calcular_indicadores_tecnicos()


📉 INDICADORES TÉCNICOS (Top 20 por Market Cap)\n


,symbol,name,precio_actual,percent_change_1h,percent_change_24h,percent_change_7d,rsi_signal,momentum_diario,tendencia_7d,volatilidad_index
0,BTC,Bitcoin,110556.7605,0.121663,-0.284796,3.526194,Neutral,-0.79,Alcista,0.41
1,ETH,Ethereum,3910.0062,0.391988,0.304552,2.150767,Neutral,-0.00,Alcista,0.09
2,USDT,Tether USDt,1.0003,0.018243,0.003325,-0.006917,Neutral,0.00,Bajista,0.01
3,BNB,BNB,1108.4156,0.484386,-1.553071,2.798715,Neutral,-1.95,Alcista,2.04
4,XRP,XRP,2.4789,-0.238282,2.736704,7.636651,Neutral,1.65,Alcista Fuerte,2.97
5,SOL,Solana,191.7677,0.373437,-0.081428,4.202122,Neutral,-0.68,Alcista,0.45
6,USDC,USDC,1.0000,0.004680,0.009077,0.000886,Neutral,0.01,Alcista,0.00
7,DOGE,Dogecoin,0.1960,0.350424,-0.303803,5.797486,Neutral,-1.13,Alcista Fuerte,0.65
8,TRX,TRON,0.3043,0.079565,-3.678345,-1.665837,Neutral,-3.44,Bajista,3.76
9,ADA,Cardano,0.6497,0.077205,-0.054271,3.469138,Neutral,-0.55,Alcista,0.13


\n📊 RESUMEN DE SEÑALES:
rsi_signal
Neutral    20
\n📈 TENDENCIAS:
tendencia_7d
Alcista           9
Alcista Fuerte    6
Bajista           4
Bajista Fuerte    1


,symbol,name,precio_actual,percent_change_1h,percent_change_24h,percent_change_7d,rsi_signal,momentum_diario,tendencia_7d,volatilidad_index
0,BTC,Bitcoin,110556.7605,0.121663,-0.284796,3.526194,Neutral,-0.79,Alcista,0.41
1,ETH,Ethereum,3910.0062,0.391988,0.304552,2.150767,Neutral,-0.00,Alcista,0.09
2,USDT,Tether USDt,1.0003,0.018243,0.003325,-0.006917,Neutral,0.00,Bajista,0.01
3,BNB,BNB,1108.4156,0.484386,-1.553071,2.798715,Neutral,-1.95,Alcista,2.04
4,XRP,XRP,2.4789,-0.238282,2.736704,7.636651,Neutral,1.65,Alcista Fuerte,2.97
5,SOL,Solana,191.7677,0.373437,-0.081428,4.202122,Neutral,-0.68,Alcista,0.45
6,USDC,USDC,1.0000,0.004680,0.009077,0.000886,Neutral,0.01,Alcista,0.00
7,DOGE,Dogecoin,0.1960,0.350424,-0.303803,5.797486,Neutral,-1.13,Alcista Fuerte,0.65
8,TRX,TRON,0.3043,0.079565,-3.678345,-1.665837,Neutral,-3.44,Bajista,3.76
9,ADA,Cardano,0.6497,0.077205,-0.054271,3.469138,Neutral,-0.55,Alcista,0.13


---

## 🤖 13. Machine Learning - Clasificación Básica

Clasificamos criptomonedas como "buena" o "mala" inversión basándonos en métricas actuales.

In [18]:
# Sistema de scoring simple para clasificar inversiones

def clasificar_inversiones():
    """
    Clasifica criptomonedas usando un sistema de scoring basado en múltiples factores.
    """
    
    query_scoring = """
    SELECT 
        symbol,
        name,
        ROUND(price, 4) as precio,
        market_cap_category,
        percent_change_24h,
        percent_change_7d,
        ROUND(volume_24h / NULLIF(market_cap, 0), 4) as liquidity_ratio,
        
        -- Sistema de puntos (0-100)
        (
            -- Rendimiento reciente (30 puntos)
            CASE 
                WHEN percent_change_7d > 20 THEN 30
                WHEN percent_change_7d > 10 THEN 25
                WHEN percent_change_7d > 0 THEN 15
                WHEN percent_change_7d > -10 THEN 5
                ELSE 0
            END +
            
            -- Liquidez (25 puntos)
            CASE 
                WHEN volume_24h / NULLIF(market_cap, 0) > 0.3 THEN 25
                WHEN volume_24h / NULLIF(market_cap, 0) > 0.15 THEN 20
                WHEN volume_24h / NULLIF(market_cap, 0) > 0.05 THEN 10
                ELSE 5
            END +
            
            -- Estabilidad market cap (25 puntos)
            CASE 
                WHEN market_cap_category = 'Mega-cap' THEN 25
                WHEN market_cap_category = 'Large-cap' THEN 20
                WHEN market_cap_category = 'Mid-cap' THEN 12
                ELSE 5
            END +
            
            -- Momentum reciente (20 puntos)
            CASE 
                WHEN percent_change_24h > 5 AND percent_change_7d > 10 THEN 20
                WHEN percent_change_24h > 0 AND percent_change_7d > 0 THEN 15
                WHEN ABS(percent_change_24h) < 5 THEN 10
                ELSE 5
            END
        ) as investment_score,
        
        -- Clasificación final
        CASE 
            WHEN (
                CASE 
                    WHEN percent_change_7d > 20 THEN 30
                    WHEN percent_change_7d > 10 THEN 25
                    WHEN percent_change_7d > 0 THEN 15
                    WHEN percent_change_7d > -10 THEN 5
                    ELSE 0
                END +
                CASE 
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.3 THEN 25
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.15 THEN 20
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.05 THEN 10
                    ELSE 5
                END +
                CASE 
                    WHEN market_cap_category = 'Mega-cap' THEN 25
                    WHEN market_cap_category = 'Large-cap' THEN 20
                    WHEN market_cap_category = 'Mid-cap' THEN 12
                    ELSE 5
                END +
                CASE 
                    WHEN percent_change_24h > 5 AND percent_change_7d > 10 THEN 20
                    WHEN percent_change_24h > 0 AND percent_change_7d > 0 THEN 15
                    WHEN ABS(percent_change_24h) < 5 THEN 10
                    ELSE 5
                END
            ) >= 70 THEN 'Excelente'
            WHEN (
                CASE 
                    WHEN percent_change_7d > 20 THEN 30
                    WHEN percent_change_7d > 10 THEN 25
                    WHEN percent_change_7d > 0 THEN 15
                    WHEN percent_change_7d > -10 THEN 5
                    ELSE 0
                END +
                CASE 
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.3 THEN 25
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.15 THEN 20
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.05 THEN 10
                    ELSE 5
                END +
                CASE 
                    WHEN market_cap_category = 'Mega-cap' THEN 25
                    WHEN market_cap_category = 'Large-cap' THEN 20
                    WHEN market_cap_category = 'Mid-cap' THEN 12
                    ELSE 5
                END +
                CASE 
                    WHEN percent_change_24h > 5 AND percent_change_7d > 10 THEN 20
                    WHEN percent_change_24h > 0 AND percent_change_7d > 0 THEN 15
                    WHEN ABS(percent_change_24h) < 5 THEN 10
                    ELSE 5
                END
            ) >= 50 THEN 'Buena'
            WHEN (
                CASE 
                    WHEN percent_change_7d > 20 THEN 30
                    WHEN percent_change_7d > 10 THEN 25
                    WHEN percent_change_7d > 0 THEN 15
                    WHEN percent_change_7d > -10 THEN 5
                    ELSE 0
                END +
                CASE 
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.3 THEN 25
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.15 THEN 20
                    WHEN volume_24h / NULLIF(market_cap, 0) > 0.05 THEN 10
                    ELSE 5
                END +
                CASE 
                    WHEN market_cap_category = 'Mega-cap' THEN 25
                    WHEN market_cap_category = 'Large-cap' THEN 20
                    WHEN market_cap_category = 'Mid-cap' THEN 12
                    ELSE 5
                END +
                CASE 
                    WHEN percent_change_24h > 5 AND percent_change_7d > 10 THEN 20
                    WHEN percent_change_24h > 0 AND percent_change_7d > 0 THEN 15
                    WHEN ABS(percent_change_24h) < 5 THEN 10
                    ELSE 5
                END
            ) >= 30 THEN 'Moderada'
            ELSE 'Evitar'
        END as clasificacion
        
    FROM crypto_listings
    WHERE market_cap > 0 AND volume_24h > 0
    ORDER BY investment_score DESC
    LIMIT 30
    """
    
    df_clasificacion = consulta_sqlite(query_scoring)
    
    if df_clasificacion is not None:
        print("\n🤖 CLASIFICACIÓN DE INVERSIONES (Top 30 por Score)\\n")
        display(df_clasificacion)
        
        # Distribución de clasificaciones
        print("\\n📊 DISTRIBUCIÓN DE CLASIFICACIONES:")
        print(df_clasificacion['clasificacion'].value_counts().to_string())
        
        # Mejores oportunidades
        mejores = df_clasificacion[df_clasificacion['clasificacion'].isin(['Excelente', 'Buena'])].head(5)
        print("\\n⭐ TOP 5 MEJORES OPORTUNIDADES:")
        print(mejores[['symbol', 'name', 'investment_score', 'clasificacion']].to_string(index=False))
    
    return df_clasificacion

# Ejecutar
clasificar_inversiones()


🤖 CLASIFICACIÓN DE INVERSIONES (Top 30 por Score)\n


,symbol,name,precio,market_cap_category,percent_change_24h,percent_change_7d,liquidity_ratio,investment_score,clasificacion
0,PUMP,Pump.fun,0.0041,Large-cap,6.520001,14.034437,0.1845,85,Excelente
1,VIRTUAL,Virtuals Protocol,0.9035,Mid-cap,14.629248,19.971687,0.5314,82,Excelente
2,USDC,USDC,1.0000,Mega-cap,0.009077,0.000886,0.2057,75,Excelente
3,ZEC,Zcash,255.4646,Large-cap,6.987474,17.590499,0.1386,75,Excelente
4,JUP,Jupiter,0.3774,Large-cap,4.790682,11.179601,0.0502,70,Excelente
5,FDUSD,First Digital USD,0.9980,Large-cap,-0.011652,0.032012,5.7902,70,Excelente
6,MYX,MYX Finance,3.2706,Mid-cap,21.540934,14.136118,0.0812,67,Buena
7,ETH,Ethereum,3910.0062,Mega-cap,0.304552,2.150767,0.0745,65,Buena
8,USDT,Tether USDt,1.0003,Mega-cap,0.003325,-0.006917,0.6741,65,Buena
9,HYPE,Hyperliquid,39.0909,Mega-cap,-0.678144,10.464707,0.0273,65,Buena


\n📊 DISTRIBUCIÓN DE CLASIFICACIONES:
clasificacion
Buena        24
Excelente     6
\n⭐ TOP 5 MEJORES OPORTUNIDADES:
 symbol              name  investment_score clasificacion
   PUMP          Pump.fun                85     Excelente
VIRTUAL Virtuals Protocol                82     Excelente
   USDC              USDC                75     Excelente
    ZEC             Zcash                75     Excelente
    JUP           Jupiter                70     Excelente


,symbol,name,precio,market_cap_category,percent_change_24h,percent_change_7d,liquidity_ratio,investment_score,clasificacion
0,PUMP,Pump.fun,0.0041,Large-cap,6.520001,14.034437,0.1845,85,Excelente
1,VIRTUAL,Virtuals Protocol,0.9035,Mid-cap,14.629248,19.971687,0.5314,82,Excelente
2,USDC,USDC,1.0000,Mega-cap,0.009077,0.000886,0.2057,75,Excelente
3,ZEC,Zcash,255.4646,Large-cap,6.987474,17.590499,0.1386,75,Excelente
4,JUP,Jupiter,0.3774,Large-cap,4.790682,11.179601,0.0502,70,Excelente
5,FDUSD,First Digital USD,0.9980,Large-cap,-0.011652,0.032012,5.7902,70,Excelente
6,MYX,MYX Finance,3.2706,Mid-cap,21.540934,14.136118,0.0812,67,Buena
7,ETH,Ethereum,3910.0062,Mega-cap,0.304552,2.150767,0.0745,65,Buena
8,USDT,Tether USDt,1.0003,Mega-cap,0.003325,-0.006917,0.6741,65,Buena
9,HYPE,Hyperliquid,39.0909,Mega-cap,-0.678144,10.464707,0.0273,65,Buena


---

## ✅ 14. Conclusiones del Análisis

### 🎯 Resumen Ejecutivo

Este proyecto demuestra un pipeline ETL completo con análisis avanzado de datos de criptomonedas:

**Logros técnicos:**
- ✅ Pipeline ETL robusto con manejo de errores y reintentos automáticos
- ✅ Transformación de JSON anidado a modelo relacional normalizado
- ✅ Base de datos SQLite con **13 tablas totales** (3 operacionales + 10 analíticas derivadas)
- ✅ Sistema de análisis multidimensional (rendimiento, liquidez, correlaciones, scoring)

**Tablas generadas:**
1. **Operacionales (3)**: `crypto_listings`, `global_metrics`, `crypto_metadata`
2. **Analíticas derivadas (10)**: `crypto_analytics`, `crypto_history`, `price_alerts`, `my_portfolio`, y otras tablas de análisis generadas durante la ejecución

**Análisis implementados:**
- 📊 Segmentación por market cap y categorías
- 📈 Time-series y tracking histórico
- 🚨 Sistema de alertas automáticas por volatilidad
- 💼 Portfolio tracking con cálculo de ROI
- 🔗 Matriz de correlación entre assets
- 🤖 Sistema de scoring ML para clasificación de inversiones

---

### 🔧 Stack Técnico Demostrado

- **ETL**: Python, pandas, requests, sqlite3
- **Análisis**: SQL avanzado (CTEs, window functions, agregaciones)
- **Métricas**: Ratios financieros, momentum, indicadores técnicos
- **Automatización**: schedule para ejecución periódica

---

### 🚀 Opciones de expansión del proyecto 

1. **Visualización**: Integrar Plotly/Matplotlib para dashboards interactivos
2. **APIs Premium**: Incorporar datos históricos OHLCV para análisis técnico completo
3. **ML Avanzado**: Modelos predictivos (LSTM, Prophet) para forecasting de precios
4. **Alertas Real-time**: Webhooks o notificaciones email/Telegram
5. **Web Dashboard**: Streamlit o Flask para interfaz web profesional

---



✨ **Uso del proyecto**: 
- Para aprender ETL y análisis de datos
- Como portfolio técnico 
- Base para proyectos más complejos de finanzas cuantitativas

---

**Proyecto completado por:** Mario Soriano Bañuls  
**Fecha:** Octubre 2025  
